![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true)
***
# *Practicum AI:* GAN - Discriminator

This exercise adapted from Baig et al. (2020) <i>The Deep Learning Workshop</i> from <a href="https://www.packtpub.com/product/the-deep-learning-workshop/9781839219856">Packt Publishers</a> (Exercise 7.03, page 338).

#### 1. Import the requisite libraries

In [ ]:
# Import the required library functions

import tensorflow as tf
import numpy as np

from numpy.random import randn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot

#### 2. Define a function to generate a real data distribution

In [2]:
def realData(loc, batch):
    '''Function to generate real samples. '''
    
    # loc is the random location or mean around which samples are centred
    # Generate numbers to right of the random point
    xr = np.arange(loc, loc + (0.1 * batch / 2), 0.1)
    xr = xr[0:int(batch / 2)]
    
    # Generate numbers to left of the random point
    xl = np.arange(loc - (0.1 * batch / 2), loc, 0.1)
    
    # Concatenating both these series 
    X1 = np.concatenate((xl, xr))
    
    # Second dependent variable
    X2 = np.sin(X1)
    
    # Reshaping both the variables and then concatenating them to an array of independent variables
    X1 = X1.reshape(batch, 1)
    X2 = X2.reshape(batch, 1)    
    X  = np.concatenate((X1, X2),axis = 1)
    
    # Generating the labels for the real data set which is 'ones'
    y = np.ones((batch, 1)) 
    
    return X,y

#### 3. Define a function to generate inputs for the generator function

In [3]:
def fakeInputs(batch, infeats):
    '''Function to generate inputs for the generator function. '''
    
    # Sample data points equal to (batch x input feature size) from a random distribution
    genInput = randn(infeats * batch)
    
    # Reshape the input 
    X = genInput.reshape(batch ,infeats)
    
    return X

#### 4. Define a function to return a generator model

In [4]:
def genModel(infeats, outfeats):
    '''Function which instantiates the generator model. '''
    
    # Define and instantiate the Generator model
    Genmodel = Sequential()
    Genmodel.add(Dense(32,activation = 'linear', kernel_initializer = 'he_uniform', input_dim = infeats))
    Genmodel.add(Dense(32,activation = 'relu', kernel_initializer = 'he_uniform'))    
    Genmodel.add(Dense(64,activation = 'elu', kernel_initializer = 'he_uniform'))    
    Genmodel.add(Dense(32,activation = 'elu', kernel_initializer = 'he_uniform'))    
    Genmodel.add(Dense(32,activation = 'selu', kernel_initializer = 'he_uniform'))
    Genmodel.add(Dense(outfeats,activation = 'selu'))
    
    return Genmodel

#### 5. Define a function to create fake samples using the generator model

In [5]:
# Function to create fake samples using the generator model
def fakedataGenerator(Genmodel, batch,infeats):
    ''' Function to create fake samples using the generator model. '''
    
    # First generate the inputs to the model
    genInputs = fakeInputs(batch,infeats)
    
    # Use these inputs inside the generator model to generate fake distribution
    X_fake = Genmodel.predict(genInputs)
    
    # Generate the labels of fake data set
    y_fake = np.zeros((batch,1))
    
    return X_fake, y_fake

#### 6. Define global parameters

In [6]:
# Define the arguments like batch size,input feature size and output feature size

batch    = 128
infeats  = 10
outfeats = 2

#### 7. Define the discriminator model

In [7]:
# Define the discriminator model

Discmodel = Sequential()
Discmodel.add(Dense(16, activation ='relu', kernel_initializer = 'he_uniform', input_dim = outfeats))
Discmodel.add(Dense(16, activation ='relu', kernel_initializer = 'he_uniform'))
Discmodel.add(Dense(16, activation ='relu', kernel_initializer = 'he_uniform'))    
Discmodel.add(Dense(1, activation ='sigmoid'))

# Compiling the model
Discmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) 

#### 8. Print a summary of the discriminator model

In [8]:
# Print the summary of the discriminator model

Discmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 609
Trainable params: 609
Non-trainable params: 0
_________________________________________________________________


#### 9. Instantiate the generator

In [9]:
# Instantiate the generator model by calling the genModel() function.

Genmodel = genModel(infeats, outfeats)
Genmodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total params: 6,722
Trainable params: 6,722
Non-trainable params: 0
____________________________________________________

#### 10. Define the number of epochs

In [22]:
# Defining the number of epochs

nEpochs = 20000

#### 11. Train the discriminator network

In [24]:
# Train the discriminator network
for i in range(nEpochs):
    
    # Generate the random number for generating real samples
    loc = np.random.normal(3,1,1)
    
    # Generate samples equal to the bath size from the real distribution
    x_real, y_real = realData(loc, batch)
    
    # Generate fake samples using the fake data generator function
    x_fake, y_fake = fakedataGenerator(Genmodel, batch, infeats)
    
    # Train the  discriminator on the real samples
    Discmodel.train_on_batch(x_real, y_real)
    
    # Train the discriminator on the fake samples
    Discmodel.train_on_batch(x_fake, y_fake)
    
    # Print the accuracy measures on the real and fake data for every 4000 epochs
    if (i) % 4000 == 0:
        # Evaluate the real distribution accuracy
        _, realAccuracy = Discmodel.evaluate(x_real, y_real, verbose = 0)
        
        # Evaluate fake distribution accuracy levels
        _, fakeAccuracy = Discmodel.evaluate(x_fake, y_fake, verbose = 0)
        
        print('Real accuracy:{R},Fake accuracy:{F}'.format(R = realAccuracy, F = fakeAccuracy))

Real accuracy:0.0,Fake accuracy:0.984375
Real accuracy:1.0,Fake accuracy:0.9609375
Real accuracy:1.0,Fake accuracy:0.96875
Real accuracy:1.0,Fake accuracy:0.984375
Real accuracy:1.0,Fake accuracy:0.96875
